In [2]:
import pandas as pd
import spacy
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt

C:\Users\hmudipalli\AppData\Local\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#!pip install --upgrade openpyxl

## Overall topics in most helpful and least helpful

In [3]:
# Load your CSV file
df = pd.read_excel("Orientation Evaluation Data_2025.xlsx", usecols=[10,12,13,18])

# Focus on the two open-ended questions
most_helpful = df.iloc[:, 1].dropna().astype(str).tolist()
least_helpful = df.iloc[:, 2].dropna().astype(str).tolist()

# Load spaCy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Preprocessing function
def preprocess(texts):
    processed_texts = []
    for doc in nlp.pipe(texts, batch_size=500):
        tokens = [token.lemma_.lower() for token in doc 
                  if token.is_alpha and not token.is_stop and len(token) > 2]
        processed_texts.append(tokens)
    return processed_texts

# Preprocess text
most_helpful_clean = preprocess(most_helpful)
least_helpful_clean = preprocess(least_helpful)

# Create dictionary and corpus
def create_lda_model(clean_texts, num_topics=5):
    dictionary = corpora.Dictionary(clean_texts)
    corpus = [dictionary.doc2bow(text) for text in clean_texts]

    lda_model = LdaModel(corpus=corpus,
                         id2word=dictionary,
                         num_topics=num_topics,
                         random_state=42,
                         update_every=1,
                         passes=10,
                         alpha='auto',
                         per_word_topics=True)
    
    return lda_model, corpus, dictionary

# Create and show topics
def display_topics(lda_model, dictionary, corpus):
    for idx, topic in lda_model.print_topics(-1):
        print(f"Topic #{idx}: {topic}")

    # Coherence score (optional)
    coherence_model = CoherenceModel(model=lda_model, texts=most_helpful_clean,
                                     dictionary=dictionary, coherence='c_v')
    print(f"\nCoherence Score: {coherence_model.get_coherence():.3f}")

# Model on 'Most Helpful'
print("\n--- Topics from 'Most Helpful' ---")
lda_most, corpus_most, dict_most = create_lda_model(most_helpful_clean)
display_topics(lda_most, dict_most, corpus_most)

# Model on 'Least Helpful'
print("\n--- Topics from 'Least Helpful' ---")
lda_least, corpus_least, dict_least = create_lda_model(least_helpful_clean)
display_topics(lda_least, dict_least, corpus_least)

# Optional: Visualize with pyLDAvis


--- Topics from 'Most Helpful' ---
Topic #0: 0.021*"like" + 0.016*"activity" + 0.016*"team" + 0.016*"find" + 0.016*"well" + 0.011*"student" + 0.011*"set" + 0.011*"allow" + 0.011*"class" + 0.011*"coverage"
Topic #1: 0.055*"helpful" + 0.027*"study" + 0.022*"session" + 0.018*"help" + 0.017*"find" + 0.016*"strategy" + 0.013*"tour" + 0.013*"different" + 0.013*"allow" + 0.013*"campus"
Topic #2: 0.027*"study" + 0.026*"feel" + 0.020*"helpful" + 0.020*"know" + 0.018*"plan" + 0.018*"different" + 0.018*"enjoy" + 0.014*"like" + 0.013*"find" + 0.013*"activity"
Topic #3: 0.020*"know" + 0.018*"think" + 0.018*"help" + 0.014*"helpful" + 0.012*"class" + 0.012*"schedule" + 0.012*"have" + 0.012*"problem" + 0.012*"day" + 0.012*"software"
Topic #4: 0.027*"help" + 0.026*"helpful" + 0.023*"know" + 0.020*"feel" + 0.019*"student" + 0.016*"time" + 0.016*"faculty" + 0.016*"presentation" + 0.016*"like" + 0.014*"start"

Coherence Score: 0.301

--- Topics from 'Least Helpful' ---
Topic #0: 0.027*"software" + 0.027*

C:\Users\hmudipalli\AppData\Local\anaconda3\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in scalar divide
  m_lr_i = np.log(numerator / denominator)
C:\Users\hmudipalli\AppData\Local\anaconda3\lib\site-packages\gensim\topic_coherence\indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in scalar divide
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))



Coherence Score: nan


In [5]:
# Optional: Visualize with pyLDAvis
import pyLDAvis.gensim_models as gensimvis
vis = gensimvis.prepare(lda_most, corpus_most, dict_most)
pyLDAvis.save_html(vis, 'lda_most_helpful.html') #pyLDAvis.show(vis)  

In [4]:
vis = gensimvis.prepare(lda_least, corpus_least, dict_least)
pyLDAvis.save_html(vis, 'lda_least_helpful.html')

In [6]:
df.columns

Index(['How would you rate your overall experience with Orientation Week? (Numerical Answer)',
       'Which part of orientation did you find most helpful and why?',
       'Which part of orientation did you find least helpful and why?',
       'What changes or improvements would you suggest for Orientation Week and why?'],
      dtype='object')

In [25]:
import pandas as pd
import spacy
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.phrases import Phrases, Phraser

# Load spaCy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Load your data
df = pd.read_excel("Orientation Evaluation Data_2025.xlsx", usecols=[10, 18])
df.columns = ['Ratings', 'Feedback']  # Rename for clarity

# Drop NaNs in relevant columns
df = df.dropna(subset=['Ratings'])

# Preprocessing function
def preprocess(texts):
    processed_texts = []
    for doc in nlp.pipe(texts, batch_size=500):
        tokens = [token.lemma_.lower() for token in doc 
                  if token.is_alpha and not token.is_stop and len(token) > 2]
        processed_texts.append(tokens)
    return processed_texts

# Step 2: Build Bigrams/Trigrams
def add_phrases(docs, min_count=2, threshold=5):
    bigram = Phrases(docs, min_count=min_count, threshold=threshold)
    bigram_mod = Phraser(bigram)
    
    trigram = Phrases(bigram_mod[docs], threshold=threshold)
    trigram_mod = Phraser(trigram)

    # Apply bigram and trigram models
    return [trigram_mod[bigram_mod[doc]] for doc in docs]

# Function to create and display topic model
def create_and_display_lda(texts, label, num_topics=5):
    clean_texts = preprocess(texts)
    # Add bigrams/trigrams
    phrased_docs = add_phrases(clean_texts)
    dictionary = corpora.Dictionary(phrased_docs)
    corpus = [dictionary.doc2bow(text) for text in phrased_docs]

    if len(dictionary) == 0:
        print(f"⚠️ Skipping '{label}' – not enough content after preprocessing.")
        return

    lda_model = LdaModel(corpus=corpus,
                         id2word=dictionary,
                         num_topics=num_topics,
                         random_state=42,
                         passes=10,
                         alpha='auto',
                         per_word_topics=False)

    print(f"\n🔹 Topics for: {label}")
    for idx, topic in lda_model.print_topics(-1):
        print(f"  Topic #{idx}: {topic}")
    
    # Coherence Score
    coherence_model = CoherenceModel(model=lda_model, texts=phrased_docs, #clean_texts
                                     dictionary=dictionary, coherence='c_v')
    print(f"  Coherence Score: {coherence_model.get_coherence():.3f}")
    
    return lda_model, dictionary, corpus

# Group by Ratings and run topic modeling
for rating_value, group in df.groupby('Ratings'):
    print(f"\n=== 📊 Rating: {rating_value} ===")

    # Most Helpful
    feedback_texts = group['Feedback'].dropna().astype(str).tolist()
    if feedback_texts:
        lda_model, dictionary, corpus = create_and_display_lda(feedback_texts, f"Feedback (Rating {rating_value})")
    
    ## visual html
    vis_feedback = gensimvis.prepare(lda_model, corpus, dictionary)
    pyLDAvis.save_html(vis_feedback, f'lda_feedback_rating_{rating_value}.html')    



=== 📊 Rating: 2 ===

🔹 Topics for: Feedback (Rating 2)
  Topic #0: 0.038*"try" + 0.038*"download" + 0.038*"time" + 0.021*"uncomfortable" + 0.021*"make" + 0.021*"lot" + 0.021*"idea" + 0.021*"difficult" + 0.021*"period" + 0.021*"get"
  Topic #1: 0.022*"download" + 0.022*"time" + 0.022*"try" + 0.022*"long" + 0.022*"people" + 0.022*"consideration" + 0.022*"virtual" + 0.022*"bonding" + 0.022*"option" + 0.022*"multiple"
  Topic #2: 0.022*"time" + 0.022*"try" + 0.022*"download" + 0.022*"sunlight" + 0.022*"consideration" + 0.022*"info" + 0.022*"uneven" + 0.022*"lighting" + 0.022*"session" + 0.022*"hour"
  Topic #3: 0.022*"time" + 0.022*"try" + 0.022*"feel" + 0.022*"lighting" + 0.022*"download" + 0.022*"consideration" + 0.022*"question" + 0.022*"food" + 0.022*"lot" + 0.022*"hour"
  Topic #4: 0.022*"download" + 0.022*"time" + 0.022*"virtual" + 0.022*"suggest" + 0.022*"try" + 0.022*"team" + 0.022*"consideration" + 0.022*"campus" + 0.022*"answering" + 0.022*"room"
  Coherence Score: 1.000

=== 📊 

In [13]:
vis_feedback = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(vis_feedback, 'lda_feedback.html')

In [26]:
lda_model.print_topics()

[(0,
  '0.045*"opportunity" + 0.034*"group" + 0.034*"roadrunner_group" + 0.024*"class" + 0.024*"small" + 0.024*"member" + 0.024*"feel" + 0.013*"improvement" + 0.013*"orientation" + 0.013*"like"'),
 (1,
  '0.031*"time" + 0.016*"tech" + 0.016*"presentation" + 0.016*"issue" + 0.016*"ahead" + 0.016*"set" + 0.016*"feel" + 0.016*"student" + 0.016*"improvement" + 0.016*"well"'),
 (2,
  '0.055*"day" + 0.034*"activity" + 0.026*"think" + 0.022*"feel" + 0.022*"early" + 0.018*"short" + 0.018*"break" + 0.017*"start" + 0.017*"bit" + 0.013*"know"'),
 (3,
  '0.027*"thursday" + 0.019*"session" + 0.019*"outside" + 0.019*"little" + 0.019*"presentation" + 0.019*"plan" + 0.019*"day" + 0.010*"help" + 0.010*"able" + 0.010*"have"'),
 (4,
  '0.024*"orientation" + 0.024*"download" + 0.018*"day" + 0.018*"time" + 0.018*"have" + 0.018*"like" + 0.012*"improvement" + 0.012*"feel" + 0.012*"provide" + 0.012*"prior"')]